## PhraseTagger nimeüksuste tuvastamiseks

Eelnevalt sai loodud loend isiku- ja organisatsiooninimeüksustest. Seda loendit saab kasutada ära järgnevalt [PhraseTaggeri ](https://github.com/estnltk/estnltk/blob/main/estnltk/estnltk/taggers/system/rule_taggers/taggers/phrase_tagger.py) töös, et märgendada huvipakkuvad nimeüksused uuesti lausetele peale. Selle tegemise mõte on esiteks tekitada kogum vaid huvipakkuvaid nimeüksusi sisaldavaid lauseid (ilma väga haruldaste nimede ja saatejuhtide nimedeta) edasises töös kasutamiseks. Teiseks võis NerTaggeril jääda esimesel korral leidmata osa nimeüksustest, mis siin huvi pakuvad ning nende nimede uuesti märgendamine aitaks neid potentsiaalselt juurde leida.

In [1]:
# Vajalikud impordid
from estnltk import Text
from estnltk.taggers.system.rule_taggers import AmbiguousRuleset, Ruleset, StaticExtractionRule
from estnltk.taggers import PhraseTagger
import csv
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json

### I PhraseTaggeri peenhäälestamine

PhraseTaggerile saab anda ette omal valikul sõnavara ja reegleid. Siin antakse sõnavaraks ette eelnevalt koostatud loend isiku- ja organisatsiooninimedest:

In [2]:
vocabulary_file = 'entity_phrases.csv'
ruleset = AmbiguousRuleset()
ruleset.load(file_name=vocabulary_file, key_column='phrase')

Samuti on soovi korral võimalik kasutada dekoraatorit kas atribuutide lisamiseks märgenduskihile või hoopis nimede seast ebasobivate käändekombinatsioonide eemaldamiseks. Siin dekoraatorit kasutusele ei võeta.

In [3]:
phrasetagger = PhraseTagger(output_layer='entity_phrases',
                      input_layer='morph_analysis',
                      input_attribute='lemma',
                      ruleset=ruleset,
                      output_attributes=['value'],
                      #decorator=decorator,
                      conflict_resolver='KEEP_MAXIMAL',
                      ignore_case=True)

Esialgu katsetan taggeri tööd mõne repliigi peal, sest tulemus peaks olema ilmekam. Hiljem märgendan sellega lauseid.

In [5]:
test_lines = []

In [6]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [7]:
for row in cur.execute("SELECT line FROM lines WHERE ID >= 6 AND ID <= 8"):
    line = json_to_text(json_text=row[0])
    test_lines.append(line)

In [8]:
con.close()

In [9]:
for line in test_lines:
    display(line)
    display(line.ner)

Text(text='Aga ma ei tea kas kaitsejõud hakkasid seletama et ei et see mingi raskeveok koos nende soomukite kes saadab sellised me lihtsalt kontrollime tammel üldse olemas on nüüd ajame korraga siis välja ka sõjaosalejatega rohkematele kommenteerib aga minu punkte siin ette et.\nMõtteks nii nagu Andrus Ansipil on selline lauset alustuse stamm fraas nagu et ma ei saa olla nõus nendega kes mõtles peaminister ma ei saa olla nõus nendega kes väidavad et nad suudavad inflatsiooni järgmise aasta esimesel poolel kuidagi kontrollijaid.\nSest vaadakem üks asi on käia aga olgu käes ja vaadata kas see.\nViisteist koma kuuskümmend neliikkagi kuidas kuidas need on eks ole soomlasel Eestis midagi osta ka teine asi on näha üks ühele võrrelda Vassel kes eurokeeles euro alus neid hindu näiteks üks mu tuttav intellektuaal Soomest ütleb et sealsed inimesed kevad Eestis nahkköitesse panema sama.\nNo kui sul on mingi armsa ja kirjatöid ja seal või või.\nKui raamat perekonna mälestused ja nahkköites ei paneks selle jutu järgmisele kontrolli Natanzis.\nHelsingis maksab neli Gent kaheksa ei uuri üks suur paks üks kõige ja meil maksab siis umbes nagu seitse juure kaheksakümmend näiteks talikummid üle suvehoidmine või näiteks üks tund.\nTa meelelahutus tegelema ütleb et kontserdi korraldab esinenud ärihelivõimenduse ja kõige selle mehe kaasa aga ma ülevedamise eurodes end lihtsalt kordades vahe noh advokaadi tunni seal kaks pool kolm korda kui kui samu asju tuleb arutada näiteks sadamas kuidas.\nSadamakaupa kus tolliterminaalist see hindade vahel lihtsalt peab tekitama olukorra kus need kes suudavad soomlasele midagi müüa teenivad teised siis ei suuda nende kõrgete hindadega eriti Tallinna hindadega väljakult väljatuleb inimesed kaasas selliselt tekib ma ei saa olla nõus nendega kes.')

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Andrus', 'Ansipil'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Eestis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Vassel'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Soomest'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Eestis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Natanzis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Helsingis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Tallinna'], [{'nertag': 'LOC'}])])

Text(text='Mul ei ole see kes kandis eneste paneb see valijad see et euro hind on pandud seal nahk nahkköites.\nKuidas nagu Taska panija kirjandust keeras nahka või pani nahka et Taska nahk köite juures olev juurde siin tähendab peab automaatselt siis üles ka kaameli siis Eesti nad alles osas mitte.')

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Taska'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Taska'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Eesti'], [{'nertag': 'LOC'}])])

Text(text='Aga küsimus ongi selles et praegu on tähelepanu sellisele kus sireenid.\nVilkuvad ja vilet saanud vilkurid seal liiguvad autode peale kuidas tuuakse raha ja see on kõik tehniline asi aga tõepoolest praegu on midagi pressiideoloogi et hinnad ei tohi muutuda et selline.\nMäletan et väiksesse printsi see oli üks kangelane kes see käskis sees kuningas see KeskEestis päikese loodud ja õhtul eks väga täpsel ajal et täiesti päikse lootes keskpäeval kunagisest et.\nSeda korraldust näitlejast näiteid et mina ei kujuta küll ette et kuidasmoodi saaks hindade sarnastumist üle üle on ka oma raha hansa seda terminit olemaskonvergents et kui seda sarnastumist ehk konvergentsi suud.\nSuudetakse vältida see tundub olevat ilmvõimatu ja teine asi kuidas see nüüd mõjust mõjutab siis inimeste siis selliste teadused kuidas end selle asemel ja siis hiljem vaatavad kui nad märkavad et et hinnahüpe siiski on toimunud hoolivat ja sellest et rahandusminister ikka vist ja päevast olin tõdeb et seda ei juhtu siin on see ometi juhtme pisut isamaa.')

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['KeskEestis'], [{'nertag': 'LOC'}])])

In [10]:
for line in test_lines:
    phrasetagger.tag(line)

In [12]:
for line in test_lines:
    display(line.text, line.ner, line.entity_phrases)

'Aga ma ei tea kas kaitsejõud hakkasid seletama et ei et see mingi raskeveok koos nende soomukite kes saadab sellised me lihtsalt kontrollime tammel üldse olemas on nüüd ajame korraga siis välja ka sõjaosalejatega rohkematele kommenteerib aga minu punkte siin ette et.\nMõtteks nii nagu Andrus Ansipil on selline lauset alustuse stamm fraas nagu et ma ei saa olla nõus nendega kes mõtles peaminister ma ei saa olla nõus nendega kes väidavad et nad suudavad inflatsiooni järgmise aasta esimesel poolel kuidagi kontrollijaid.\nSest vaadakem üks asi on käia aga olgu käes ja vaadata kas see.\nViisteist koma kuuskümmend neliikkagi kuidas kuidas need on eks ole soomlasel Eestis midagi osta ka teine asi on näha üks ühele võrrelda Vassel kes eurokeeles euro alus neid hindu näiteks üks mu tuttav intellektuaal Soomest ütleb et sealsed inimesed kevad Eestis nahkköitesse panema sama.\nNo kui sul on mingi armsa ja kirjatöid ja seal või või.\nKui raamat perekonna mälestused ja nahkköites ei paneks selle j

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Andrus', 'Ansipil'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Eestis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Vassel'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Soomest'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Eestis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Natanzis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Helsingis'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Tallinna'], [{'nertag': 'LOC'}])])

Layer(name='entity_phrases', attributes=('value',), spans=SL[EnvelopingSpan(['tammel'], [{'value': 'PER'}]),
EnvelopingSpan(['Andrus', 'Ansipil'], [{'value': 'PER'}])])

'Mul ei ole see kes kandis eneste paneb see valijad see et euro hind on pandud seal nahk nahkköites.\nKuidas nagu Taska panija kirjandust keeras nahka või pani nahka et Taska nahk köite juures olev juurde siin tähendab peab automaatselt siis üles ka kaameli siis Eesti nad alles osas mitte.'

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Taska'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Taska'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Eesti'], [{'nertag': 'LOC'}])])

Layer(name='entity_phrases', attributes=('value',), spans=SL[])

'Aga küsimus ongi selles et praegu on tähelepanu sellisele kus sireenid.\nVilkuvad ja vilet saanud vilkurid seal liiguvad autode peale kuidas tuuakse raha ja see on kõik tehniline asi aga tõepoolest praegu on midagi pressiideoloogi et hinnad ei tohi muutuda et selline.\nMäletan et väiksesse printsi see oli üks kangelane kes see käskis sees kuningas see KeskEestis päikese loodud ja õhtul eks väga täpsel ajal et täiesti päikse lootes keskpäeval kunagisest et.\nSeda korraldust näitlejast näiteid et mina ei kujuta küll ette et kuidasmoodi saaks hindade sarnastumist üle üle on ka oma raha hansa seda terminit olemaskonvergents et kui seda sarnastumist ehk konvergentsi suud.\nSuudetakse vältida see tundub olevat ilmvõimatu ja teine asi kuidas see nüüd mõjust mõjutab siis inimeste siis selliste teadused kuidas end selle asemel ja siis hiljem vaatavad kui nad märkavad et et hinnahüpe siiski on toimunud hoolivat ja sellest et rahandusminister ikka vist ja päevast olin tõdeb et seda ei juhtu siin

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['KeskEestis'], [{'nertag': 'LOC'}])])

Layer(name='entity_phrases', attributes=('value',), spans=SL[EnvelopingSpan(['isamaa'], [{'value': 'ORG'}])])

### II Lausetele uuesti nimeüksuste märgendamine

Andmebaasist loetakse sisse kõik laused, et märgendada PhraseTaggeri abil uuesti peale loendis olevad nimeüksused.

In [42]:
sentences = []
sentence_ids = []

In [43]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [44]:
for row in cur.execute("SELECT ID, sentence FROM sentences"):
    sentence_id = row[0]
    sentence = json_to_text(json_text=row[1])
    sentences.append(sentence)
    sentence_ids.append(sentence_id)

In [45]:
con.close()

Märgendatakse kõik laused:

In [46]:
for sentence in sentences:
    phrasetagger.tag(sentence)

Vaatame üht lauset, kus tagger mõne nimeüksuse tuvastas:

In [47]:
for sentence in sentences:
    if len(sentence.entity_phrases) > 0:
        display(sentence, sentence.entity_phrases)
        break

Text(text='Tere siin kuku raadio keskpäevatund Tartus Rein kilk Tallinnas Hans H luik ja priit hõbemägi ja see esimene lukus ka faktor oli lepik maani tõesti suur rahakotis pühendas nendele kolmele konteiner eurorahadele kolmene konteineritäis.')

Layer(name='entity_phrases', attributes=('value',), spans=SL[EnvelopingSpan(['keskpäevatund'], [{'value': 'ORG'}]),
EnvelopingSpan(['Rein'], [{'value': 'PER'}, {'value': 'ORG'}]),
EnvelopingSpan(['luik'], [{'value': 'PER'}]),
EnvelopingSpan(['priit'], [{'value': 'PER'}])])

### III Märgendatud lausete salvestamine

Andmebaasi luuaks uus tabel lausete jaoks, milles PhraseTagger mõne nimeüksusefraasi tuvastas.

In [48]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [49]:
cur.execute("CREATE TABLE phrase_tagger_sentences(ID INTEGER PRIMARY KEY, sentence_ID INTEGER, sentence TEXT)")

In [50]:
for i in range(len(sentences)):
    try:
        if len(sentences[i].entity_phrases) > 0:
            sentence_json = text_to_json(sentences[i])
            cur.execute("""INSERT INTO phrase_tagger_sentences
                                    (sentence_ID, sentence)
                                    VALUES (?, ?);""", (sentence_ids[i], sentence_json))
    except:
        continue
    con.commit()

In [51]:
con.close()